In [ ]:
import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass, field 

In [ ]:
@dataclass 
class Holiday:
    name: str 
    date: datetime 

    def __str__(self):
        return f'{self.name} is on {self.date}'

In [ ]:
weekdays = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3,
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6
}

In [ ]:
@dataclass
class HolidayList: 
    holidays: list = field(default_factory=list)

    def addHoliday(self, holidayObj):
        if isinstance(holidayObj, Holiday):
            self.holidays.append(holidayObj)
            print("The Holiday was added to the list")
        else:
            print("Error: That is not a Holiday object")
        pass
    def findHoliday(self, name, date):
        for holiday in self.holidays:
            if holiday.name == name and holiday.date == date:
                return holiday 
            return None 
        pass
    # I may have to play around with the indentation of return and pass
    
    def removeHoliday(self, name, date):
        holiday = self.findHoliday(name, date)
        if holiday:
            self.holidays.remove(holiday)
            print(f'{name} on {date} was removed from the list.')
        else:
            print(f'No holiday with that name ({name}) and date ({date}) was found in the lsit.')
        pass
    def read_json(self, filelocation):
        with open('holidays.json', 'r') as f:
            data = json.load(f)
        for holiday in data:
            holiday_obj = Holiday(holiday['name'], holiday['date'])
            self.addHoliday(holiday_obj) 
        pass
    def save_to_json(self, filelocation):
        data = [{'name': holiday.name, 'date': holiday.date} for holiday in self.holidays]
        with open('holidays.json', 'w') as f:
            json.dump(data,f)
        pass
    def scrapeHolidays(self, trs, soup, tag, date, holidays, year):
        current_year = year 
        try:
            for i in range(7, len(trs)):
                trs = soup.find_all('th')
                instance = trs[i]
                partial_date = instance.contents
                date = f'{current_year}--{date}'
               

                holiday = Holiday(name, date)
                # if holiday not in holidays:
                if not any(holiday.name == name and holiday.date == date for holiday in holidays):
                    self.addHoliday(date)
            

            for i in range (128, 691):
                tag = soup.find_all('a')[i]
                name = tag.contents

                holiday = Holiday(name, date)
                if holiday not in holidays:
                    self.addHoliday(name)
        except:
            print("An error ocurred while scraping and inserting the holidays")
        
    def numHolidays(self):
        return len(self.holidays)
    
    def filter_holidays_by_week(year, week_number, holidays): 
        filtered_holidays = []
        for holiday in holidays:
            # creating datetime object for the holiday
            holiday_date = datetime(holiday.year, holiday.month, holiday.day)
            

             # If the holiday's week number matches the input week number(Monday = 0 .....), add it to the filtered list
            # if holiday_date.weekday() == weekdays[week_number]:
            if holiday_date.isocalender()[1] == week_number:
                filtered_holidays.append(holiday)

        return filtered_holidays
        
    def displayHolidaysInWeek(year, week_number):
        holidays = filter_holidays_by_week( year, week_number, holidays)
        for holiday in holidays:
            print(holiday)

    # def getWeather(weekNum):
    
    # Convert weekNum to range between two days
    # Use Try / Except to catch problems
    # Query API for weather in that week range
    # Format weather information and return weather string.
    def viewCurrentWeek():
        now = datetime.datetime.now()
        current_year = now.year
    # I want to return the week number here. I'm pretty sure that is what the 
    # '%U' does
        current_week_number = now.strftime('%U')
        holidays = filter_holidays_by_week(current_year, current_week_number)
        displayHolidaysInWeek(year, week_number, holidays)
        # this function runs when I put it and the previous two methods in a 
        # separate cell, but I get an error saying that the filter holidays method
        # is not defined (at the bottom)

    def main():
        holidays = HolidayList()
        holidays.read_json("holidays.json")
        holidays.scrapeHolidays()
        while True:
            print("1. Add a Holiday")
            print("2. Remove a Holiday")
            print("3. Save Holiday List")
            print("4. View Holidays")
            print("5. Exit")

            action = input('Enter your choice: ')
            if not action.isdigit() or not 1 <= int(action) <=5:
                print('Invalid choice. Please try again')
                continue 

            if action == "1":
                holidays.addHoliday()
            elif action == "2":
                holidays.removeHoliday()
            elif action == "3":
                holidays.save_to_json()
            elif action == "4":
                holidays.filter_holidays_by_week()
            else:
                break

        # Ask the User if they would like to Continue, if not, end the while loop
            continue_choice = input("Would you like to continue? (y/n) ")
            if continue_choice.lower() != "y":
                break

    if __name__ == "__main__":
        main()



    

    


    